In [9]:
import os
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit 
import itertools
from sklearn.metrics import *

In [2]:
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")
sub = pd.read_csv("sample_submission.csv")

In [3]:
def fe(data):
    
    df = pd.DataFrame()
    data['totl_anglr_vel'] = (data['angular_velocity_X']**2 + data['angular_velocity_Y']**2 +
                             data['angular_velocity_Z']**2)** 0.5
    data['totl_linr_acc'] = (data['linear_acceleration_X']**2 + data['linear_acceleration_Y']**2 +
                             data['linear_acceleration_Z'])**0.5
    data['totl_xyz'] = (data['orientation_X']**2 + data['orientation_Y']**2 +
                             data['orientation_Z'])**0.5
   
    data['acc_vs_vel'] = data['totl_linr_acc'] / data['totl_anglr_vel']
    
    for col in data.columns:
        if col in ['row_id','series_id','measurement_number']:
            continue
        df[col + '_mean'] = data.groupby(['series_id'])[col].mean()
        df[col + '_median'] = data.groupby(['series_id'])[col].median()
        df[col + '_max'] = data.groupby(['series_id'])[col].max()
        df[col + '_min'] = data.groupby(['series_id'])[col].min()
        df[col + '_std'] = data.groupby(['series_id'])[col].std()
        df[col + '_range'] = df[col + '_max'] - df[col + '_min']
        df[col + '_maxtoMin'] = df[col + '_max'] / df[col + '_min']
        df[col + '_mean_abs_chg'] = data.groupby(['series_id'])[col].apply(lambda x: np.mean(np.abs(np.diff(x))))
        df[col + '_abs_max'] = data.groupby(['series_id'])[col].apply(lambda x: np.max(np.abs(x)))
        df[col + '_abs_min'] = data.groupby(['series_id'])[col].apply(lambda x: np.min(np.abs(x)))
        df[col + '_abs_avg'] = (df[col + '_abs_min'] + df[col + '_abs_max'])/2
    return df

In [4]:
X_train = fe(X_train)
X_test = fe(X_test)
print(X_train.shape)

(3810, 154)


In [5]:
le = LabelEncoder()
y_train['surface'] = le.fit_transform(y_train['surface'])

In [14]:
X_train.fillna(0, inplace = True)
X_test.fillna(0, inplace = True)
X_train.replace(-np.inf, 0, inplace = True)
X_train.replace(np.inf, 0, inplace = True)
X_test.replace(-np.inf, 0, inplace = True)
X_test.replace(np.inf, 0, inplace = True)

In [26]:
def k_folds(X, y, X_test, k):
    folds = StratifiedShuffleSplit(n_splits = k, test_size=0.1, random_state=2019)
    y_test = np.zeros((X_test.shape[0], 9))
    y_oof = np.zeros((X.shape[0]))
    score = 0
    for i, (train_idx, val_idx) in  enumerate(folds.split(X, y)):
        clf =  RandomForestClassifier(n_estimators = 500, n_jobs = -1)
        clf.fit(X_train.iloc[train_idx], y[train_idx])
        y_oof[val_idx] = clf.predict(X.iloc[val_idx])
        y_test += clf.predict_proba(X_test) / folds.n_splits
        score += clf.score(X.iloc[val_idx], y[val_idx])
        print('Fold: {} score: {}'.format(i,clf.score(X.iloc[val_idx], y[val_idx])))
    print('Avg Accuracy', score / folds.n_splits) 
        
    return y_oof, y_test 

In [27]:
y_oof, y_test = k_folds(X_train, y_train['surface'], X_test, k= 50)

Fold: 0 score: 0.8818897637795275
Fold: 1 score: 0.884514435695538
Fold: 2 score: 0.905511811023622
Fold: 3 score: 0.89501312335958
Fold: 4 score: 0.889763779527559
Fold: 5 score: 0.9028871391076115
Fold: 6 score: 0.8923884514435696
Fold: 7 score: 0.8871391076115486
Fold: 8 score: 0.889763779527559
Fold: 9 score: 0.8923884514435696
Fold: 10 score: 0.89501312335958
Fold: 11 score: 0.9081364829396326
Fold: 12 score: 0.8608923884514436
Fold: 13 score: 0.8871391076115486
Fold: 14 score: 0.868766404199475
Fold: 15 score: 0.863517060367454
Fold: 16 score: 0.8923884514435696
Fold: 17 score: 0.8976377952755905
Fold: 18 score: 0.910761154855643
Fold: 19 score: 0.889763779527559
Fold: 20 score: 0.8923884514435696
Fold: 21 score: 0.8923884514435696
Fold: 22 score: 0.8792650918635171
Fold: 23 score: 0.8792650918635171
Fold: 24 score: 0.9002624671916011
Fold: 25 score: 0.9002624671916011
Fold: 26 score: 0.9133858267716536
Fold: 27 score: 0.8818897637795275
Fold: 28 score: 0.8818897637795275
Fold: 2

In [28]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_oof,y_train['surface'])

array([[154,   5,   0,   0,   4,  10,   4,   5,   2],
       [ 15, 695,  20,   1,  12,  17,   5,  25,  12],
       [  0,  10, 287,   0,   3,   3,   0,   2,   3],
       [  0,   0,   0,  10,   0,   0,   0,   0,   0],
       [  1,  11,   1,   0, 278,   5,   0,   3,   1],
       [  5,  15,  11,   0,   3, 666,   5,   8,  23],
       [  4,  10,   3,   2,   0,  10, 278,   5,   2],
       [  2,  12,   9,   0,   0,   3,   0, 458,  19],
       [  8,  21,  32,   8,   8,  18,   5,   8, 545]])

In [29]:
y_test = np.argmax(y_test, axis=1)
sub['surface'] = le.inverse_transform(y_test)
sub.to_csv('submission_rf_new5.csv', index=False)